## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
# data_dir = "OrfPathHealth"
data_dir = "bookwise"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir Loader
"""
documents = PyPDFDirectoryLoader(inputdirectory).load()
## File Loader
# documents = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf").load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
"""

loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.76s/it]

Number of chunks =  4
Page Title Speak Effectively in Sales Communication

Page Contents Summarized Successfully communicating with customers is the foundation for all sales Two tactics to increase customer reception: understand their language and focus on them Understand their language: Avoid one-size-fits-all sales pitch, recognize customers' unique language based on life experiences Tailor your approach and language for each customer Focus on them, not you: Avoid talking about your company and product features Turn the spotlight on customers: Discuss their problems, values, and purchasing plans

Page Title Innovate with Less: Strategies for Tough Times

Page Contents Summarized Even large corporations need to innovate as start-ups do when resources and time become scarce. Forget the big budget: Innovation doesn't have to be expensive. Use open-source software, online market research tools, and virtual prototypes. Test in the real market: Launch a 'good enough' design quickly and ref

## Create a dataframe of all the chunks

In [3]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(4, 3)


,text,source,chunk_id
0,Book Title\n\nAtomic habits\n\nAuthor\n\nJames...,data_input/bookwise/book_library.txt,a205c3080db9433aa3990e9173974e4f
1,Book Title\n\nManagement Tips\n\nAuthor\n\nHar...,data_input/bookwise/book_library.txt,f58590bad09a4c769457feac2da54840
2,Page Title Manage Stress by Facing It\n\nPage ...,data_input/bookwise/book_library.txt,5762081865084547849cee1fa1e0a484
3,Page Title Speak Effectively in Sales Communic...,data_input/bookwise/book_library.txt,2a8b990a43d94c8b9f85fb5fe4301084


## Extract Concepts

In [4]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2ConceptsList
from helpers.df_helpers import concepts2Df

In [5]:
concepts_list = df2ConceptsList(df[10:12])

An error occurred: Out of range float values are not JSON compliant
None


ERROR ### Here is the buggy response:  None 


Series([], dtype: float64)
Series([], dtype: float64)


In [11]:
dfne = concepts2Df(concepts_list)
dfne.head()

NameError: name 'concepts_list' is not defined

### Write CSV to an output directory

Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of names entities

        df = dataframe of chunks

In [28]:
if not os.path.exists(outputdirectory):
   os.makedirs(outputdirectory)
   
dfne.to_csv(outputdirectory/"concepts.csv", sep="|", index=False)
df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)

## Named Entities from Concepts

**Not using this right now**

Extracting named entities our of concepts. 


In [3]:
from transformers import pipeline

ner = pipeline("token-classification", model="2rtl3/mn-xlm-roberta-base-named-entity", aggregation_strategy="simple")
# ner = pipeline("token-classification", model="dslim/bert-large-NER", aggregation_strategy="simple")

def row2NamedEntities(row):
    ner_results = ner(row['entity'])
    metadata = {'chunk_id': row['chunk_id'], 'type': 'entity'}
    entities = []
    for result in ner_results:
        entities = entities + [{'entity': result['word'], 'catetory': result['entity_group'], **metadata}]
        
    return entities



def dfText2DfNE(dataframe: pd.DataFrame):
    ## Takes a dataframe from the parsed data and returns dataframe with named entities. 
    ## The input dataframe must have a entity and a chunk_id column. 

    ## 1. Calculate named entities for each row of the dataframe. 
    results = dataframe.apply(row2NamedEntities, axis=1).reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities. 
    entities_list = np.concatenate(results).ravel().tolist()

    ## Remove all NaN entities
    entities_dataframe = pd.DataFrame(entities_list).replace(' ', np.nan)
    entities_dataframe = entities_dataframe.dropna(subset=['entity'])

    ## Count the number of occurances per chunk id
    # entities_dataframe = entities_dataframe.groupby(['entity', 'category', 'chunk_id']).size().reset_index(name='count')

    return entities_dataframe

In [5]:

dataframe_dir = 'OrfPathHealth'
df_concepts = pd.read_csv(f"./data/output/{dataframe_dir}/concepts.csv", sep="|")

dfc_split = dfText2DfNE(df_concepts)
dfc_split

,entity,catetory,chunk_id,type
0,Mental Health,MISC,83d4d0367bb0467e811782a4ada3bbb9,entity
1,Health Equity,ORG,83d4d0367bb0467e811782a4ada3bbb9,entity
2,World Health Organization (WHO),ORG,83d4d0367bb0467e811782a4ada3bbb9,entity
3,United Nations (UN),ORG,83d4d0367bb0467e811782a4ada3bbb9,entity
4,Sustainable Development Goals (SDGs),MISC,83d4d0367bb0467e811782a4ada3bbb9,entity
...,...,...,...,...
967,National University of Singapore's Saw Swee Ho...,ORG,4dbae5e4a3ee45bdbf9d55dec8921c2c,entity
968,Associate Professor,MISC,4dbae5e4a3ee45bdbf9d55dec8921c2c,entity
969,Jeremy Lim,PER,4dbae5e4a3ee45bdbf9d55dec8921c2c,entity
970,Janice Tan,PER,4dbae5e4a3ee45bdbf9d55dec8921c2c,entity
